In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [2]:
# для google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# розмір картинки
IMG_SIZE = (75, 75)

# папка з датасетом у якій підпапки з фотографіями кожного символа 
# назви класів будуть відповідати іменам підпапок
directory = "images/"

In [4]:
data_generator = ImageDataGenerator(validation_split = 0.2, rescale = 1.0 / 255)
 
# датасет для тренування
train = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'training')

# датасет для тестування
test = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'validation')

Found 286 images belonging to 29 classes.
Found 55 images belonging to 29 classes.


In [5]:
def createModel(image_shape = IMG_SIZE):
    '''
    Аргументи:
        image_shape -- ширина та висота картинки
    Повертає:
        tf.keras.model
    '''
    input_shape = image_shape + (3,)
    
    base_model = InceptionV3(input_shape = input_shape,
                            include_top = False,
                            weights = "imagenet",
                            classes = 29,
                            classifier_activation = "softmax")
    # заморозимо шари, щоб їх не тренувати
    base_model.trainable = False

    # вхідний шар
    inputs = tf.keras.Input(shape = input_shape) 

    # для нормалізації інпута [-1;1]
    x = tf.keras.applications.inception_v3.preprocess_input(inputs) 

    x = base_model(x, training = False) 
    x = tfl.GlobalAveragePooling2D()(x) 
    x = tfl.Dropout(0.2)(x)
    outputs = tfl.Dense(29, activation = "softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [6]:
model = createModel(IMG_SIZE)

Metal device set to: Apple M1


2022-11-07 23:30:34.489165: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-07 23:30:34.489265: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# метрики
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision_f1 = precision(y_true, y_pred)
    recall_f1 = recall(y_true, y_pred)
    return 2 * ((precision_f1 * recall_f1) / (precision_f1 + recall_f1 + K.epsilon()))

In [8]:
base_learning_rate = 0.01

initial_epochs = 10

loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(base_learning_rate)
metrics = ['acc', f1_score, precision, recall]

In [9]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [10]:
history = model.fit(train,
                    validation_data = test,
                    epochs = initial_epochs)

Epoch 1/10


2022-11-07 23:30:36.272355: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2022-11-07 23:30:37.459662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/9 [=========================>....] - ETA: 0s - loss: 4.7884 - acc: 0.3031 - f1_score: 0.2967 - precision: 0.4265 - recall: 0.2385         

2022-11-07 23:30:39.040076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 3s 206ms/step - loss: 4.8868 - acc: 0.2972 - f1_score: 0.2955 - precision: 0.4162 - recall: 0.2398 - val_loss: 1.6719 - val_acc: 0.6727 - val_f1_score: 0.6494 - val_precision: 0.7143 - val_recall: 0.5978
Epoch 2/10
9/9 [==============================] - 1s 59ms/step - loss: 1.8331 - acc: 0.6748 - f1_score: 0.7088 - precision: 0.7794 - recall: 0.6509 - val_loss: 0.3571 - val_acc: 0.9091 - val_f1_score: 0.8985 - val_precision: 0.9524 - val_recall: 0.8505
Epoch 3/10
9/9 [==============================] - 1s 56ms/step - loss: 0.7450 - acc: 0.8427 - f1_score: 0.8554 - precision: 0.8990 - recall: 0.8176 - val_loss: 0.2812 - val_acc: 0.9273 - val_f1_score: 0.9288 - val_precision: 0.9389 - val_recall: 0.9192
Epoch 4/10
9/9 [==============================] - 1s 55ms/step - loss: 0.4282 - acc: 0.9021 - f1_score: 0.9105 - precision: 0.9343 - recall: 0.8884 - val_loss: 0.2757 - val_acc: 0.9455 - val_f1_score: 0.9651 - val_precision: 0.9839 - val_recall: 0.947

In [11]:
# model.summary()

# print("Кількість шарів у базовій моделі: ", len(tunedModel.layers))

In [12]:
tunedModel = model.layers[3]
tunedModel.trainable = True

fine_tune_start = 250

In [13]:
# Заморозимо всі шари перед fine_tune_start
for layer in tunedModel.layers[:fine_tune_start]:
    layer.trainable = False

optimizer = Adam(learning_rate = 0.0001 * base_learning_rate)

fine_tune_epochs = 100
total_epochs =  initial_epochs + fine_tune_epochs

In [14]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [15]:
history_fine = model.fit(train,
                         validation_data = test,
                         epochs = total_epochs,
                         initial_epoch = history.epoch[-1])

Epoch 10/110


2022-11-07 23:30:45.743802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - ETA: 0s - loss: 0.0541 - acc: 0.9720 - f1_score: 0.9763 - precision: 0.9816 - recall: 0.9713

2022-11-07 23:30:47.972442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 4s 270ms/step - loss: 0.0541 - acc: 0.9720 - f1_score: 0.9763 - precision: 0.9816 - recall: 0.9713 - val_loss: 0.1235 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 11/110
9/9 [==============================] - 1s 90ms/step - loss: 0.0423 - acc: 0.9860 - f1_score: 0.9859 - precision: 0.9859 - recall: 0.9859 - val_loss: 0.1227 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 12/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0372 - acc: 0.9965 - f1_score: 0.9948 - precision: 0.9965 - recall: 0.9931 - val_loss: 0.1240 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 13/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0415 - acc: 0.9930 - f1_score: 0.9928 - precision: 0.9928 - recall: 0.9928 - val_loss: 0.1264 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall:

9/9 [==============================] - 1s 81ms/step - loss: 0.0186 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1407 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 43/110
9/9 [==============================] - 1s 82ms/step - loss: 0.0120 - acc: 1.0000 - f1_score: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.1411 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 44/110
9/9 [==============================] - 1s 96ms/step - loss: 0.0256 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1421 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 45/110
9/9 [==============================] - 1s 85ms/step - loss: 0.0251 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1435 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 

9/9 [==============================] - 1s 83ms/step - loss: 0.0161 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1502 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 75/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0164 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1502 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 76/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0192 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1500 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 77/110
9/9 [==============================] - 1s 91ms/step - loss: 0.0231 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1501 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 

9/9 [==============================] - 1s 81ms/step - loss: 0.0171 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1562 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 107/110
9/9 [==============================] - 1s 81ms/step - loss: 0.0146 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1562 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 108/110
9/9 [==============================] - 1s 81ms/step - loss: 0.0124 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1567 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 109/110
9/9 [==============================] - 1s 81ms/step - loss: 0.0147 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1561 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recal

In [16]:
model.save('Fine_tuned_model01.h5', overwrite = True, include_optimizer = True)

In [ ]:
model.